In [1]:


import nibabel as nib
import nilearn as nil
from nilearn import plotting

from socket import gethostname
from yaml.loader import SafeLoader
import yaml

from level2_utils import read_yaml_for_host



print(gethostname())
# Open the file and load the file
config = read_yaml_for_host('l2_config.yml')

Benjamins-MacBook-Pro-2.local
Benjamins-MacBook-Pro-2.local


In [3]:
config.keys()

dict_keys(['rubric_dir', 'dropbox_datapath', 'nonbids_data_path', 'sst_wave1_path', 'fmriprep_dir', 'spm_l2_script_template_filepath', 'spm_l2_script_w_confounders_template_filepath', 'confounder_template_path', 'confounder_consess_template_path', 'confounder_conspec_template_path', 'input_mat_path', 'dev_scripts_path', 'sst_level_2_path', 'spm_path', 'nii_raw_path', 'mask_location', 'template_mlr_main', 'template_mlr_confounder_no_centering', 'template_mlr_consess', 'template_mlr_conspec'])

## Striatum joint mask

In [26]:
#get the paths of the six striatal mask areas
import glob


mask_path = config['mask_location']

striatal_masks = glob.glob(mask_path + 'striatum' + '/*.nii.gz')

#load the masks and add them
striatal_joint_mask = nil.image.math_img(
    " + ".join(["img" + str(i) for i in range(len(striatal_masks))]),
    **{'img'+str(i):sm for i, sm in enumerate(striatal_masks)}
    )

#now save that mask
striatal_joint_mask_filepath = mask_path + 'aim3/striatum_joint_mask.nii.gz'
striatal_joint_mask.to_filename(striatal_joint_mask_filepath)


Next mask we want is an intersection of the striatum mask and the motor control striatal mask

In [29]:
motor_control_mask = nil.image.math_img(
    "striatum * motor_control",
    striatum = striatal_joint_mask_filepath,
    motor_control = mask_path + 'aim3/neurosynth/motor control_association-test_z_FDR_0.01.nii.gz')

motor_control_mask.to_filename(mask_path + 'aim3/motor_control_striatum_joint_mask.nii.gz')

## Striatal SST functional contrast

Before we run this, the striatal clusters need to be manually created in SPM.

In [ ]:
dev_scripts_abs_path = config['dev_scripts_path']
posterror_folder = 'posterror_cues_no_rt_20230512'

#these two files need to be manually created in SPM.
# roi file to open
roi_files = [dev_scripts_abs_path + 'fMRI/fx/models/SST/level2/' + posterror_folder + '/CueFollowing(CS>FS)/CueFollowing(CS>FS)striatal_cluster_1.nii',
dev_scripts_abs_path + 'fMRI/fx/models/SST/level2/' + posterror_folder + '/CueFollowing(CS>FS)/CueFollowing(CS>FS)striatal_cluster_2.nii']
# load each of the roi files
 
roi_data = []
for roi_file in roi_files:
    print(roi_file)
    ni_file = nib.load(roi_file)
    #binarize roi using nilearn
    ni_file = nil.image.math_img("img > 0", img=ni_file)
    roi_data = roi_data + [ni_file]
    #display the roi
    #plotting.plot_roi(ni_file, title=roi_file)
    print("displayed")
#combine the rois
#concatenate them
combined_roi_data = nil.image.concat_imgs(roi_data)
#then add them
combined_roi_data = nil.image.math_img("np.sum(imgs, axis=3)", imgs=combined_roi_data)

plotting.plot_roi(combined_roi_data, title="combined")
#save the combined roi
combined_roi_data.to_filename(
    dev_scripts_abs_path + '/fMRI/fx/models/SST/level2/' + posterror_folder + '/CueFollowing(CS>FS)/CueFollowing(CS>FS)striatal_cluster_combined.nii')

    




## Hedy Kober image conversion

This converts a mask from hdr/img format to nii format

nil.load

In [78]:
koban_kober_img = nib.load(config['mask_location'] + "aim3/signatures/koban_kober_craving_wmapN99_boot10K_02-May-2022.img")

img_variance = koban_kober_img.get_fdata()[koban_kober_img.get_fdata()!=0].var()
variance_reciprocal = 1/img_variance
img_not_zero = np.sum(koban_kober_img.get_fdata()!=0)

In [84]:
np.sum(koban_kober_img.get_fdata()!=0)

69160

In [54]:
koban_kober_img_var1 = nil.image.math_img("img *" + str(variance_reciprocal), img=koban_kober_img)

In [57]:
nib.save(koban_kober_img_var1, config['mask_location'] + "aim3/signatures/koban_kober_craving_wmapN99_boot10K_02-May-2022_var1.nii.gz")

In [86]:
koban_kober_img_scaled_up = nil.image.math_img("img *" + str(img_not_zero), img=koban_kober_img)
nib.save(koban_kober_img_scaled_up, config['mask_location'] + "aim3/signatures/koban_kober_craving_wmapN99_boot10K_02-May-2022_notzero.nii.gz")

## Response inhibition

In [4]:
mask_path = config['mask_location']


In [5]:
mask_path

'/Users/benjaminsmith/Google Drive/oregon/data/DEV/brainmaps/'

In [28]:
#get the paths of the six striatal mask areas
import glob


mask_path = config['mask_location']

neurosynth_response_inhibition_filepath = mask_path + 'response_inhibition_related/response inhibition_association-test_z_FDR_0.01.nii.gz'

superior_frontal_gyrus_filepath = mask_path + 'response_inhibition_related/harvardoxford-cortical_prob_Superior Frontal Gyrus.nii.gz'

right_hemisphere_filepath = mask_path + 'harvardoxford-subcortical_prob_Right Cerebral Cortex.nii.gz'

frontal_pole_filepath  = mask_path + 'response_inhibition_related/harvardoxford-cortical_prob_Frontal Pole.nii.gz'

#set a threshold of 25 for the h-o masks
superior_frontal_gyrus_25 = nil.image.math_img("img > 25", img=superior_frontal_gyrus_filepath)
frontal_pole_25 = nil.image.math_img("img > 25", img=frontal_pole_filepath)
right_hemisphere_0 = nil.image.math_img("img > 0", img=right_hemisphere)


In [10]:
neurosynth_response_inhibition = nil.image.load_img(neurosynth_response_inhibition_filepath)

In [21]:
#could use the regions extractor, but I would rather use the neurosynth mask as, although we don't necessarily get continguous regions,
#the strict multiply method gets us a more replicable mask
# import nilearn.regions as regions

# regions_list, index_of_each_map = regions.connected_regions(neurosynth_response_inhibition, min_region_size=1000)
# regions_list.to_filename(mask_path + 'response_inhibition_related/neurosynth_response_inhibition_connected_regions.nii.gz')

In [ ]:
# functional_mask_filepath=neurosynth_response_inhibition_filepath
# base_anatomical_img_set=superior_frontal_gyrus_filepath
# hemisphere_filepath = right_hemisphere
# functional_thresh = 5
# out_filename = mask_path + 'response_inhibition_anatomical_combined/neurosynth_response_inhibition_right_sup_fron_gyrus.nii.gz'

# multiply the two h-o masks by the hemisphere mask to get the right hemisphere
def combine_neurosynth_anatomical(functional_mask_filepath, base_anatomical_img_set, hemisphere_filepath, functional_thresh, out_filename):
    base_anatomical_img = nil.image.load_img(base_anatomical_img_set)
    hemisphere_img = nil.image.load_img(hemisphere_filepath)
    # binarize the h-o masks
    anatomical_mask_bin_25 = nil.image.binarize_img(base_anatomical_img,25)
    anatomical_mask_bin_25.to_filename(superior_frontal_gyrus_filepath.replace('.nii.gz', '_bin_25.nii.gz'))
    # binarize the hemisphere mask
    hemisphere_mask_bin_25 = nil.image.binarize_img(hemisphere_img,25)
    hemispheric_anatomical_mask = nil.image.math_img("img1 * img2", img1=anatomical_mask_bin_25, img2=hemisphere_mask_bin_25)
    hemispheric_anatomical_mask.to_filename(superior_frontal_gyrus_filepath.replace('.nii.gz', '_hemi_bin_25.nii.gz'))
    # binarize the neurosynth mask
    functional_mask_bin = nil.image.binarize_img(functional_mask_filepath,functional_thresh)
    # multiply the neurosynth mask by each of the h-o masks
    combined_mask = nil.image.math_img("img1 * img2", img1=functional_mask_bin, img2=hemispheric_anatomical_mask)
    # save each output
    combined_mask.to_filename(out_filename)

combine_neurosynth_anatomical(neurosynth_response_inhibition_filepath,superior_frontal_gyrus_filepath,
                              right_hemisphere_filepath,functional_thresh,out_filename)

In [ ]:
#load the masks and add them
motor_control_mask = nil.image.math_img(
    "striatum * motor_control",
    striatum = striatal_joint_mask_filepath,
    motor_control = mask_path + 'aim3/neurosynth/motor control_association-test_z_FDR_0.01.nii.gz')

#now save that mask
striatal_joint_mask_filepath = mask_path + 'aim3/striatum_joint_mask.nii.gz'
striatal_joint_mask.to_filename(striatal_joint_mask_filepath)
